In [25]:
import numpy as np
import os, shutil
import time
import matplotlib.pyplot as plt


from keras.models import load_model

import torchsummary

import torch.nn.functional as F


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

## Preparation des données et fonction fit

In [2]:
#Chargement du jeu de données
original_dataset_dir = r'C:\Users\pauli\Documents\M2\ML et DL\projet\Git\DamageDetection'

train_damage_dir = original_dataset_dir + '/train_another/damage'
validation_damage_dir = original_dataset_dir + '/validation_another/damage'
test_damage_dir = original_dataset_dir + '/test/damage'

train_nodamage_dir = original_dataset_dir + '/train_another/no_damage'
validation_nodamage_dir = original_dataset_dir + '/validation_another/no_damage'
test_nodamage_dir = original_dataset_dir + '/test/no_damage'

print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))


############################################################################################################

#Definir la fonction de transformation
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Définir le jeu de données d'entraînement
train_dataset = ImageFolder(original_dataset_dir + '/train_another', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

# Définir le jeu de données de validation
val_dataset = ImageFolder(original_dataset_dir + '/validation_another', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=True)


total training damage images:  5000
total validation damage images:  1000
total test damage images:  1000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [7]:
# fonction pour entrainer les modeles
def fit(model, criterion, optimizer, train_loader, val_loader, epochs, steps_per_epoch, validation_steps):
    epoch_nums = []
    training_acc = []
    validation_acc = []
    training_loss = []
    validation_loss = []
    
    for epoch in range(epochs):
        accuracy = 0.0
        running_loss = 0.0
        start_time = time.time()
        # Entraîne le modèle sur les données d'entraînement
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            # Entraîne le modèle sur un lot de données
            outputs = model(images)
            loss = criterion(outputs, labels.float().view(-1,1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            accuracy += (torch.round(outputs) == labels.float().view(-1,1)).sum().item() / len(labels)
            running_loss += loss.item()
        
            if (i+1) % steps_per_epoch == 0:
                break
        train_loss = running_loss / steps_per_epoch
        train_acc = accuracy / steps_per_epoch 
                
        
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_acc = 0
            for i, (images, labels) in enumerate(val_loader):
                outputs = model(images)
                val_loss += criterion(outputs, labels.float().view(-1,1)).item()
                _, predicted = torch.max(outputs.data, 1)
                val_acc += (torch.round(outputs) == labels.float().view(-1,1)).sum().item() / len(labels)
                
                if i >= validation_steps:
                    break
        
        val_loss /= validation_steps
        val_acc /= validation_steps 

        epoch_nums.append(epoch+1)
        training_loss.append(train_loss)
        validation_loss.append(val_loss)
        training_acc.append(train_acc)
        validation_acc.append(val_acc)
        
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        print(f"Epoch [{epoch+1}/{epochs}], Time : {elapsed_time:.4f}, Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
        
    return {
        'training_acc': training_acc,
        'validation_acc': validation_acc,
        'training_loss': training_loss,
        'validation_loss': validation_loss
    }

def params_to_update(model) :
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
    return params_to_update

## Avec le modèle pré-entrainé resnet18


In [10]:
class ResNet18Sigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet18(weights=True)
        self.model.fc = nn.Linear(512, 1)

    def forward(self, x):
        x = self.model(x)
        x = nn.Sigmoid()(x)
        return x

model1 = ResNet18Sigmoid()

#torchsummary.summary(model1, (3,150,150))

In [11]:
#Resnet18 avec l'optimizer RMSProp
#train_losses1, val_accuracies1, val_losses1 = train_model(optimizer1, criterion, model1, val_loader, train_loader)
criterion = nn.BCELoss()
optimizer = optim.RMSprop(params_to_update(model1), lr=0.001)
history = fit(model1, criterion, optimizer, train_loader, val_loader, 5, 100, 50 )

KeyboardInterrupt: 

In [ ]:
training_acc = history['training_acc']
validation_acc = history['validation_acc']
training_loss = history['training_loss']
validation_loss = history['validation_loss']
epoch_nums = range(1,len(training_acc)+1)

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#Resnet18 avec l'optimizer Adam
criterion = nn.BCELoss()
optimizer = optim.Adam(params_to_update(model1), lr=0.001)
history = fit(model1, criterion, optimizer, train_loader, val_loader, 5, 100, 50 )

In [ ]:
training_acc = history['training_acc']
validation_acc = history['validation_acc']
training_loss = history['training_loss']
validation_loss = history['validation_loss']
epoch_nums = range(1,len(training_acc)+1)

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

 ### Modification de plusieurs couches de ResNet18
 

In [28]:
class ResNet18Sigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet18(weights=True)
        #num_features = self.model.fc.in_features
        self.fc = nn.Linear(1000, 256)
        self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = F.relu(self.fc(x))
        x = F.relu(self.fc2(x))        
        x = self.sigmoid(x)
        return x

model1 = ResNet18Sigmoid()



In [29]:
#Resnet18  modifié avec l'optimizer Adam
criterion = nn.BCELoss()
optimizer = optim.Adam(params_to_update(model1), lr=0.001)
history = fit(model1, criterion, optimizer, train_loader, val_loader, 5, 100, 50 )

KeyboardInterrupt: 

In [ ]:
training_acc = history['training_acc']
validation_acc = history['validation_acc']
training_loss = history['training_loss']
validation_loss = history['validation_loss']
epoch_nums = range(1,len(training_acc)+1)

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Avec le modèle pré-entrainé Inception_v3

In [46]:
class Inception3Sigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.inception_v3(weights=True)
        self.conv_layer = nn.Conv2d(3, 32, (3,3))
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = F.relu(self.conv_layer(x))
        x = self.sigmoid(x)
        return x

model2 = Inception3Sigmoid()



RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6193152 bytes.

In [47]:
#Inception_v3 avec l'optimizer Adam
criterion = nn.BCELoss()
optimizer = optim.Adam(params_to_update(model2), lr=0.001)
history = fit(model2, criterion, optimizer, train_loader, val_loader, 5, 100, 50 )

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5400000 bytes.

In [ ]:
training_acc = history['training_acc']
validation_acc = history['validation_acc']
training_loss = history['training_loss']
validation_loss = history['validation_loss']
epoch_nums = range(1,len(training_acc)+1)

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Avec le modèle pré-entrainé VGG16

In [17]:
class VGG16Sigmoid(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.model = torchvision.models.vgg16(weights=True)
        self.model.classifier[6] = nn.Linear(4096, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = self.sigmoid(x)
        return x

model3 = VGG16Sigmoid()


In [18]:
#VGG16 avec l'optimizer Adam
criterion = nn.BCELoss()
optimizer = optim.Adam(params_to_update(model3), lr=0.001)
history = fit(model3, criterion, optimizer, train_loader, val_loader, 5, 100, 50 )

KeyboardInterrupt: 

In [ ]:
training_acc = history['training_acc']
validation_acc = history['validation_acc']
training_loss = history['training_loss']
validation_loss = history['validation_loss']
epoch_nums = range(1,len(training_acc)+1)

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()